## ** An Experimental Study on Performance of Non-intrusive Load Monitoring **

- Author : Gyubaek Kim (database.kim@yonsei.ac.kr)
- Description
  * Various algorithms have been utilized by NILM systems. The existing methods adopted neural network-based algorithms. The neural 
  network-based algorithms performed better than combinational optimization and hidden Markov models. Thus, we compared the 
  performance of the neural network-based algorithms and decision tree-based ensemble algorithms, such as random forest and gradient 
  boosting machine (GBM).
- License
   * All rights reserved. No part of this code may be reproduced or transmitted in any form or by any means, or utilized by any information storage and retrieval system without written permission from the copyright owner. You can use this source code for free projects only. I will not allow commercial use of this code. I am not allowing anyone to modify any or all parts of the source code for commercial purposes. By using my source code, you agree to the following: 
       * 1. You will not distribute any or all parts of this source code for commercial use 
       * 2. You will cite me as the original creator of this source code. 
       * 3. You will inform me of its use before you use any or all parts of this source code. 
   For use of any or all parts of this source code, email me. This source code is provided free to use for everybody provided this is used non-commercially. Violators of this agreement will be subject to legal action by the author. If you see commercial applications or software using any or all parts of this source code, inform me.  

#### import packages of benchmark project (contrib. project) to be compared

- https://github.com/OdysseasKr/neural-disaggregator
- The project implemented 5 neural network-based NILM models
  * RNN (Recurrent Neural Network)
  * GRU (Gated Recurrent Unit)
  * Window GRU
  * DAE (Denosing Autoencoder)
  * S2P (Short Sequence to Point)

In [1]:
#### NIMLTK as common framework ####
from nilmtk.utils import find_nearest
from nilmtk.feature_detectors import cluster
from nilmtk.legacy.disaggregate import Disaggregator
from nilmtk.datastore import HDFDataStore
from nilmtk import DataSet

#### Benchmark Resource ####
from benchmark.rnndisaggregator import RNNDisaggregator
from benchmark.grudisaggregator import GRUDisaggregator
from benchmark.windowgrudisaggregator import WindowGRUDisaggregator
from benchmark.daedisaggregator import DAEDisaggregator
from benchmark.shortseq2pointdisaggregator import ShortSeq2PointDisaggregator
from benchmark.metrics import recall_precision_accuracy_f1, relative_error_total_energy, mean_absolute_error

from work.benchmark.rnndisaggregator_fe import RNNDisaggregatorFE
from work.benchmark.grudisaggregator_fe import GRUDisaggregatorFE
from work.benchmark.windowgrudisaggregator_fe import WindowGRUDisaggregatorFE
from work.benchmark.daedisaggregator_fe import DAEDisaggregatorFE
from work.benchmark.shortseq2pointdisaggregator_fe import ShortSeq2PointDisaggregatorFE

#### Our Model ####
from work.dtdisaggregator import DTDisaggregator
from work.utils import checkSimilarity

import pandas as pd
import numpy as np
import h5py
import numpy.random
import time
import datetime
import math
import itertools
import warnings

from scipy.signal import find_peaks

from matplotlib import rcParams
import matplotlib.pyplot as plt

numpy.random.seed(42)
warnings.filterwarnings('ignore')

%matplotlib inline
rcParams['figure.figsize'] = (23, 10)

/home/kgbmanse/anaconda3/envs/nilm/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


#### Configuration

In [2]:
###############################################################
# Common
###############################################################
TRAIN_SAMPLE_PERIOD = 1
TEST_SAMPLE_PERIOD = 1

EPOCHS = 1

DATA_FILE = 'data/enertalk.h5'

T_S = '2017-01-01'
T_E = '2017-01-20'
P_S = '2017-01-21'
P_E = '2017-01-31'

HOME_ID = 1
#appliances = ['television', 'washing machine', 'rice cooker', 'water filter', 'microwave']
appliances = ['rice cooker', 'water filter', 'microwave']

In [3]:
# start_time, end_time : YYYY-MM-DD HH:MI:SS
def disaggregate(dataset, model, appliance, start_time, end_time, evaluate=False):
    
    dataset.set_window(start=start_time, end=end_time)
    
    test_elec = dataset.buildings[HOME_ID].elec    
    test_mains = test_elec.all_meters()[0]   
    test_meter = test_elec.submeters()[appliance]
    
    tempfile = 'work/temp/' + str(time.time())
    
    output = HDFDataStore(tempfile, 'w')    
    model.disaggregate(test_mains, output, test_meter, sample_period=TEST_SAMPLE_PERIOD)
    output.close()
    
    result = DataSet(tempfile)    
    res_elec = result.buildings[HOME_ID].elec

    predicted = res_elec[appliance] 
    
    ground_truth = None
   
    if evaluate == True:
        
        ground_truth = test_elec[appliance]

        # Performance Metric
        print("============ Relative error in total energy: {}".format(relative_error_total_energy(predicted, ground_truth)))
        print("============ Mean absolute error(in Watts): {}".format(mean_absolute_error(predicted, ground_truth)))
        
        # Performance Metric
        rpaf = recall_precision_accuracy_f1(predicted, ground_truth)
        print("============ Recall: {}".format(rpaf[0]))
        print("============ Precision: {}".format(rpaf[1]))
        print("============ Accuracy: {}".format(rpaf[2]))
        print("============ F1 Score: {}".format(rpaf[3]))
        
    return predicted

## Alogorithm Comparison

- Description
  In the first experiemnt, only total electricity consumption was used as training data

In [4]:
dataset = DataSet(DATA_FILE)
dataset.set_window(start=T_S, end=T_E)

train_elec = dataset.buildings[HOME_ID].elec
train_mains = train_elec.all_meters()[0] # The aggregated meter that provides the input

for aname in appliances:
    
    train_meter = train_elec.submeters()[aname]
    
    models = {}
    
    models['GRU-' + aname] = GRUDisaggregator()
    models['RNN-' + aname] = RNNDisaggregator()
    models['WGRU-' + aname] = WindowGRUDisaggregator(window_size=50)
    models['DAE-' + aname] = DAEDisaggregator(256)
    models['S2P-' + aname] = ShortSeq2PointDisaggregator(window_size=100)
    models['DRF-' + aname] = DTDisaggregator(algorithm='DRF', use_feature=False, reactive=False)
    models['GBM-' + aname] = DTDisaggregator(algorithm='GBM', use_feature=False, reactive=False)
    
    for mname, model in models.items():
        
        try:
            
            print("*"*20)
            print("[" + mname + "]") 
            print("*" *20)

            start = time.time()
            model.train(train_mains, train_meter, epochs=EPOCHS, sample_period=TRAIN_SAMPLE_PERIOD)
            end = time.time()            
            print("elapsed time (secs) = " + str(end-start))
            
            disaggregate(dataset, model, aname, P_S, P_E, True)
        
        except Exception as e:
            print('train & test error', e)


2021-12-06 23:53:15.303399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 50, 16)            80        
_________________________________________________________________
bidirectional_4 (Bidirection (None, 50, 128)           31488     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               198144    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_17 (Conv1D)           (None, 100, 30)           330       
_________________________________________________________________
dropout_20 (Dropout)         (None, 100, 30)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 100, 30)           7230      
_________________________________________________________________
dropout_21 (Dropout)         (None, 100, 30)           0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 100, 40)           7240      
_________________________________________________________________
dropout_22 (Dropout)         (None, 100, 40)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 100, 50)          

********************
[GRU-microwave]
********************
6750/6750 [==============================] - 72s 10ms/step - loss: 3.5234e-04
elapsed time (secs) = 92.21284818649292
New sensible chunk: 357621
New sensible chunk: 173735
New sensible chunk: 332127
============ Relative error in total energy: 0.1963919748254595
============ Mean absolute error(in Watts): 9.82852894660954
============ Recall: 0.1265758029743852
============ Precision: 0.25933967208205433
============ Accuracy: 0.921721497056681
============ F1 Score: 0.17012081327963852
********************
[RNN-microwave]
********************
6750/6750 [==============================] - 111s 16ms/step - loss: 4.0738e-04
elapsed time (secs) = 132.09389972686768
New sensible chunk: 357621
New sensible chunk: 173735
New sensible chunk: 332127
============ Relative error in total energy: 0.01373852945409226
============ Mean absolute error(in Watts): 11.520309423795906
============ Recall: 0.43179742025066686
============ Precision